# Task 2 : Restaurant Recommendation

<h3>Objective :</h3>

- Create a restaurant recommendation system based on user preferences.

<h3>Steps :</h3>

- Preprocess the dataset by handling missing values and encoding categorical variables.
- Determine the criteria for restaurant recommendations (e.g., cuisine preference, price range).
- Implement a content-based filtering approach where users are recommended restaurants similar to their preferred criteria.
- Test the recommendation system by providing sample user preferences and evaluating the quality of recommendations.

## Import necessary Libraries and Data Loading

In [1]:
# Import Libraries
import pandas as pd

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Ignore warnings

import warnings
warnings.filterwarnings('ignore') 

In [4]:
# Data Loading (CSV file)

data_df = pd.read_csv(r"E:\Cognify\Dataset .csv")
data_df.head(5)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


## Data Analysis

In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [ ]:
# Handle missing values in Cuisines column

data_df['Cuisines'].fillna('Unknown', inplace=True)

<ipython-input-7-bcca33c02fd2>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df['Cuisines'].fillna('Unknown', inplace=True)


In [ ]:
df = data_df

## Feature Engineering

In [ ]:
# Convert categorical columns into numerical format

binary_columns = ['Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu']
for col in binary_columns:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

In [ ]:
# One-hot encode Cuisines (splitting multiple cuisines per restaurant)

df['Cuisines'] = df['Cuisines'].apply(lambda x: x.split(',')[0])  # Keep primary cuisine
df = pd.get_dummies(df, columns=['Cuisines'])

In [ ]:
# Normalize numerical features

scaler = MinMaxScaler()
df[['Average Cost for two', 'Aggregate rating', 'Votes']] = scaler.fit_transform(df[['Average Cost for two', 'Aggregate rating', 'Votes']])

In [ ]:
# Drop unnecessary columns

df.drop(columns=['Restaurant Name', 'Address', 'Locality', 'Locality Verbose', 'Rating color', 'Rating text', 'Currency'], inplace=True)

In [ ]:
# Display the processed dataset info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Columns: 133 entries, Restaurant ID to Cuisines_World Cuisine
dtypes: bool(120), float64(5), int64(7), object(1)
memory usage: 2.0+ MB


In [ ]:
df.head()

,Restaurant ID,Country Code,City,Longitude,Latitude,Average Cost for two,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,...,Cuisines_Tea,Cuisines_Tex-Mex,Cuisines_Thai,Cuisines_Tibetan,Cuisines_Turkish,Cuisines_Turkish Pizza,Cuisines_Unknown,Cuisines_Vietnamese,Cuisines_Western,Cuisines_World Cuisine
0,6317637,162,Makati City,121.027535,14.565443,0.001375,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,6304287,162,Makati City,121.014101,14.553708,0.001500,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,6300002,162,Mandaluyong City,121.056831,14.581404,0.005000,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,6318506,162,Mandaluyong City,121.056475,14.585318,0.001875,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,6314302,162,Mandaluyong City,121.057508,14.584450,0.001875,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False


## Data Spliting

In [ ]:
# Extract features and target (Restaurant ID for recommendation)

features = df.drop(columns=['Restaurant ID', 'City'])
target = df['Restaurant ID']

In [ ]:
# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

## Model Building

In [ ]:
# Define the deep learning model

def model(input_shape):
  input_layer = Input(shape=(input_shape,))
  dense1 = Dense(128, activation='relu')(input_layer)
  dense2 = Dense(64, activation='relu')(dense1)
  dense3 = Dense(32, activation='relu')(dense2)
  output_layer = Dense(1, activation='linear')(dense3)  # Prediction of Restaurant ID

  # Compile the model
  model = Model(inputs=input_layer, outputs=output_layer)
  model.compile(optimizer='adam', loss='mse', metrics=['mae'])

  return model

In [ ]:
# Train the model
train_model = model(X_train.shape[1])
train_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=1)

Epoch 1/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 154988676382720.0000 - mae: 8814981.0000 - val_loss: 152804232527872.0000 - val_mae: 8830299.0000
Epoch 2/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 143789045841920.0000 - mae: 8735079.0000 - val_loss: 100032741113856.0000 - val_mae: 8458864.0000
Epoch 3/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 93610330554368.0000 - mae: 8529400.0000 - val_loss: 86232902139904.0000 - val_mae: 8775846.0000
Epoch 4/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 85914839678976.0000 - mae: 8764838.0000 - val_loss: 84735074238464.0000 - val_mae: 8762669.0000
Epoch 5/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 86057672507392.0000 - mae: 8782539.0000 - val_loss: 83729414356992.0000 - val_mae: 8702772.0000
Epoch 6/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 85084124217344.0000 - mae: 8735901.0000 - val_loss: 82952058830848.0000 - val_mae: 8642087.0000
Epoch 7/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 

In [ ]:
# Function to get restaurant recommendations based on cuisine

def recommend_restaurants_dl(cuisine, num_recommendations=5):
    cuisine_column = f"Cuisines_{cuisine}"
    if cuisine_column not in df.columns:
        return f"No restaurants found for cuisine: {cuisine}"

    # Get restaurants serving the cuisine
    cuisine_df = df[df[cuisine_column] == 1].copy()
    if cuisine_df.empty:
        return f"No restaurants found for cuisine: {cuisine}"

    # Predict restaurant IDs
    predictions = train_model.predict(cuisine_df.drop(columns=['Restaurant ID', 'City']))
    cuisine_df['Predicted_Score'] = predictions.flatten()

    # Get top recommended restaurants
    recommended_restaurants = cuisine_df.sort_values(by='Predicted_Score', ascending=False).head(num_recommendations)
    return recommended_restaurants[['Restaurant ID', 'City']]

In [ ]:
recommendations = recommend_restaurants_dl('Desserts', num_recommendations=5)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [ ]:
# print(recommendations)

## Restaurant Recommendatiom

In [ ]:
# Retrieve restaurant names based on the recommended restaurant IDs


restaurant_names = data_df[data_df['Restaurant ID'].isin(recommendations['Restaurant ID'])][['Restaurant ID', 'Restaurant Name', 'City']]
restaurant_names

,Restaurant ID,Restaurant Name,City
131,17294441,Giuseppe's Pizza & Italian Specialities,Augusta
277,17342548,Happy Joe's Pizza & Ice Cream,Dubuque
410,17582682,Thai Paradise,Pocatello
511,17099856,Datz,Tampa Bay
543,17696891,Four Queens Dairy Cream,Waterloo
